<a href="https://colab.research.google.com/github/luferIPCA/MIA-MLA-23-24/blob/master/2_Data_Manipulation_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masters' in Applied Artificial Intelligence
## Machine Learning Algorithms Course (MLA)

Notebooks for MLA course

by [*lufer*](mailto:lufer@ipca.pt)

(2024)

---

# Part II - Datasets Manipulation (I)

This is the first notebook for datasets manipulation. Cleaning, Normalizing, Initializing are some of the required tasks during dataset preparation for training.

**Contents**:

1. **Features Engineering**
2. **Cleaning Data**
3. **Outliers**



## Environment preparation


### Importing necessary Libraries

In [1]:
import pandas as pd
import numpy as np

Mounting Drive

In [2]:

from google.colab import drive

# it will ask for your google drive credentiaals
drive.mount('/content/gDrive/', force_remount=True)

Mounted at /content/gDrive/


*Loading dataset*

In [3]:
#global path variable
path="/content/gDrive/MyDrive/Colab Notebooks/MIA - ML - 2024-2025/Datasets/"
#path

In [4]:
import requests

download_url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv"
target_csv_path = path+"nbaAll.csv"

#create a local file with remote csv data
response = requests.get(download_url)
response.raise_for_status()
with open(target_csv_path, "wb") as f:
    f.write(response.content)
print("Download ready.")

nbaOriginal = pd.read_csv(path+"nbaAll.csv")

Download ready.


In [5]:
nba = nbaOriginal.copy()


## 1 - Features Engineering

Features Engineering means a set of actions to deal with features (columns) of a dataset.

It involves selecting, manipulating and transforming raw data into features that can be used in training models.

**Feature Engineering Definition**

*Feature engineering is the process of selecting, manipulating and transforming raw data into features that can be used in supervised learning. It consists of five processes: feature creation, transformations, feature extraction, exploratory data analysis and benchmarking.*

Processes involving:


*   Feature creation (adding or removing some features)
*   Transformations
*   Feature extraction
*   Exploratory data analysis
*   Benchmark

Concrete actions:

*   Inputation (missing values: categorical and numerical)
*   Handling outliers (Removing, Replacing values, Discretization)
*   Log Transform (handle confusing data)
*   One-Hot Encoding (unique value for each possible case)
*   Scaling (Normalization, Standardization )



### Warming...

Analysing the dataset

In [ ]:
#checking dataset structure
nba.shape

(126314, 23)

In [ ]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   gameorder      126314 non-null  int64  
 1   game_id        126314 non-null  object 
 2   lg_id          126314 non-null  object 
 3   _iscopy        126314 non-null  int64  
 4   year_id        126314 non-null  int64  
 5   date_game      126314 non-null  object 
 6   seasongame     126314 non-null  int64  
 7   is_playoffs    126314 non-null  int64  
 8   team_id        126314 non-null  object 
 9   fran_id        126314 non-null  object 
 10  pts            126314 non-null  int64  
 11  elo_i          126314 non-null  float64
 12  elo_n          126314 non-null  float64
 13  win_equiv      126314 non-null  float64
 14  opp_id         126314 non-null  object 
 15  opp_fran       126314 non-null  object 
 16  opp_pts        126314 non-null  int64  
 17  opp_elo_i      126314 non-nul

In [6]:
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN


*Filtering columns with "isin"*

method *isin()* checks if the Dataframe contains the specified value(s)

In [ ]:
#get values in the period 1948-1949
nbaYear = nba[nba["year_id"].isin([1948, 1949])]
nbaYear

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
700,351,194711120WSC,NBA,0,1948,11/12/1947,1,0,WSC,Capitols,...,50.173782,BLB,Baltimore,55,1419.0000,1414.4618,H,W,0.717674,NaN
701,351,194711120WSC,NBA,1,1948,11/12/1947,1,0,BLB,Baltimore,...,42.867378,WSC,Capitols,63,1481.0703,1485.6085,A,L,0.282326,NaN
702,352,194711130BLB,NBA,0,1948,11/13/1947,2,0,BLB,Baltimore,...,43.288319,BOS,Celtics,74,1289.9966,1285.9733,H,W,0.784507,NaN
703,352,194711130BLB,NBA,1,1948,11/13/1947,1,0,BOS,Celtics,...,29.691414,BLB,Baltimore,85,1414.4618,1418.4851,A,L,0.215493,NaN
704,353,194711130CHS,NBA,0,1948,11/13/1947,1,0,CHS,Stags,...,45.430969,STB,Bombers,70,1395.4702,1389.6464,H,W,0.688478,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,943,194904090WSC,NBA,1,1949,4/9/1949,68,1,MNL,Lakers,...,59.256477,WSC,Capitols,83,1469.5483,1484.5758,A,L,0.606789,NaN
1886,944,194904110WSC,NBA,0,1949,4/11/1949,70,1,WSC,Capitols,...,46.550896,MNL,Lakers,65,1629.8854,1618.4757,H,W,0.435161,NaN
1887,944,194904110WSC,NBA,1,1949,4/11/1949,69,1,MNL,Lakers,...,58.284595,WSC,Capitols,74,1484.5758,1495.9855,A,L,0.564839,NaN
1888,945,194904130LAL,NBA,0,1949,4/13/1949,70,1,MNL,Lakers,...,58.820530,WSC,Capitols,56,1495.9855,1489.7295,H,W,0.782579,at Saint Paul MN


*Get first N columns from a dataframe*

In [ ]:
n=3
aux = nba.iloc[:,:n]
aux
#question: how to check the first n rows?

,gameorder,game_id,lg_id
0,1,194611010TRH,NBA
1,1,194611010TRH,NBA
2,2,194611020CHS,NBA
3,2,194611020CHS,NBA
4,3,194611020DTF,NBA
...,...,...,...
126309,63155,201506110CLE,NBA
126310,63156,201506140GSW,NBA
126311,63156,201506140GSW,NBA
126312,63157,201506170CLE,NBA


*Get last N columns from a dataframe*

In [ ]:
n=3
aux = nba.iloc[:,-n:]
aux
#question: how to check the last n rows?

,game_result,forecast,notes
0,L,0.640065,NaN
1,W,0.359935,NaN
2,W,0.631101,NaN
3,L,0.368899,NaN
4,L,0.640065,NaN
...,...,...,...
126309,L,0.546572,NaN
126310,W,0.765565,NaN
126311,L,0.234435,NaN
126312,L,0.481450,NaN


### Deriving new Feature

(this will be more explored later on *Categorical to Numerical* section)

*Create new Feature (column)*

In [7]:
#convert object to datetime value
nba["date_played"] = pd.to_datetime(nba["date_game"])
#nba
#nba.columns

*Create new Feature from calculus over others*

In [ ]:
#See https://www.plus2net.com/python/pandas-dt-timedelta64.php
import datetime
from datetime import date
today = pd.to_datetime(date.today())
#new column
nba['DaysCPassed'] = (today-nba['date_played']) / np.timedelta64(1, 'D')
nba.shape

(126314, 26)

In [ ]:
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes,date_played,DaysPassed,DaysCPassed
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,68,1300.0000,1306.7233,H,L,0.640065,NaN,1946-11-01,28555.0,28555.0
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,66,1300.0000,1293.2767,A,W,0.359935,NaN,1946-11-01,28555.0,28555.0
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,47,1306.7233,1297.0712,H,W,0.631101,NaN,1946-11-02,28554.0,28554.0
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,63,1300.0000,1309.6521,A,L,0.368899,NaN,1946-11-02,28554.0,28554.0
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,50,1300.0000,1320.3811,H,L,0.640065,NaN,1946-11-02,28554.0,28554.0


In [ ]:
nba.DaysCPassed.max()

28555.0

Separate datetime in columns for month, day, year

In [22]:
#check
nba["date_played"]

,date_played
0,1946-11-01
1,1946-11-01
2,1946-11-02
3,1946-11-02
4,1946-11-02
...,...
126309,2015-06-11
126310,2015-06-14
126311,2015-06-14
126312,2015-06-16


In [ ]:
#check
nba.dtypes

In [10]:
#create new columns year, month and day
nba['year'] = nba["date_played"].dt.year
nba['month'] = nba["date_played"].dt.month
nba['day'] = nba["date_played"].dt.dayofyear

In [11]:
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes,date_played,year,month,day
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,1300.0000,1306.7233,H,L,0.640065,NaN,1946-11-01,1946,11,305
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,1300.0000,1293.2767,A,W,0.359935,NaN,1946-11-01,1946,11,305
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,1306.7233,1297.0712,H,W,0.631101,NaN,1946-11-02,1946,11,306
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,1300.0000,1309.6521,A,L,0.368899,NaN,1946-11-02,1946,11,306
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,1300.0000,1320.3811,H,L,0.640065,NaN,1946-11-02,1946,11,306


Create new column for Holydays (boolean)

In [17]:
import holidays
#print(dir(holidays))
# Create a dict-like object for Portugues holidays
pt_holidays = holidays.Portugal()
#show all
for feriado in pt_holidays['2020-01-01': '2020-12-31'] :
    print(feriado)


['ABW', 'AD', 'AE', 'AGO', 'AL', 'ALB', 'AM', 'AND', 'AO', 'APR', 'AR', 'ARE', 'ARG', 'ARM', 'ARMED_FORCES', 'AS', 'ASM', 'AT', 'AU', 'AUG', 'AUS', 'AUT', 'AW', 'AZ', 'AZE', 'Albania', 'Algeria', 'AmericanSamoa', 'Andorra', 'Angola', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'B3', 'BA', 'BAH', 'BANK', 'BB', 'BD', 'BDI', 'BE', 'BEL', 'BF', 'BFA', 'BG', 'BGD', 'BH', 'BHS', 'BI', 'BIH', 'BLG', 'BLR', 'BLZ', 'BN', 'BO', 'BOL', 'BR', 'BRA', 'BRB', 'BRN', 'BS', 'BVMF', 'BW', 'BWA', 'BY', 'BZ', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Bolivia', 'BosniaAndHerzegovina', 'Botswana', 'BrasilBolsaBalcao', 'Brazil', 'Brunei', 'Bulgaria', 'BurkinaFaso', 'Burundi', 'CA', 'CAN', 'CATHOLIC', 'CG', 'CH', 'CHE', 'CHINESE', 'CHL', 'CHN', 'CHRISTIAN', 'CL', 'CM', 'CMR', 'CN', 'CO', 'COG', 'COL', 'CR', 'CRI', 'CU', 'CUB', 'CUW', 'CW', 'CY', 'CYP', 'CZ', 'CZE', 'Cambodia', 'Cameroon', 'Canada', 'Chad', 'Chile', 'China', 'Colombia', 'Congo',

In [21]:
# Create a function to check if the birthdate is a holiday
def is_holiday(birthdate):
    # Use only the month and day of the birthdate for comparison
    holiday_dates = {date for date in pt_holidays if date.month == birthdate.month and date.day == birthdate.day}
    return bool(holiday_dates)

# Apply the function to create the "Holiday" column
nba["Holiday"] = nba["date_played"].apply(is_holiday)
nba.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_elo_n,game_location,game_result,forecast,notes,date_played,year,month,day,Holiday
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,1306.7233,H,L,0.640065,NaN,1946-11-01,1946,11,305,True
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,1293.2767,A,W,0.359935,NaN,1946-11-01,1946,11,305,True
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,1297.0712,H,W,0.631101,NaN,1946-11-02,1946,11,306,False
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,1309.6521,A,L,0.368899,NaN,1946-11-02,1946,11,306,False
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,1320.3811,H,L,0.640065,NaN,1946-11-02,1946,11,306,False


### Change features names

In [ ]:
renamedNba = nba.rename(columns={"DaysCPassed": "DaysPassed"})

In [ ]:
renamedNba.info()
print('-'*50)
nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   gameorder      126314 non-null  int64         
 1   game_id        126314 non-null  object        
 2   lg_id          126314 non-null  object        
 3   _iscopy        126314 non-null  int64         
 4   year_id        126314 non-null  int64         
 5   date_game      126314 non-null  object        
 6   seasongame     126314 non-null  int64         
 7   is_playoffs    126314 non-null  int64         
 8   team_id        126314 non-null  object        
 9   fran_id        126314 non-null  object        
 10  pts            126314 non-null  int64         
 11  elo_i          126314 non-null  float64       
 12  elo_n          126314 non-null  float64       
 13  win_equiv      126314 non-null  float64       
 14  opp_id         126314 non-null  object        
 15  

### Deleting Features

*Delete a particular Feature (column)*

In [ ]:
renamedNba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   gameorder      126314 non-null  int64         
 1   game_id        126314 non-null  object        
 2   lg_id          126314 non-null  object        
 3   _iscopy        126314 non-null  int64         
 4   year_id        126314 non-null  int64         
 5   date_game      126314 non-null  object        
 6   seasongame     126314 non-null  int64         
 7   is_playoffs    126314 non-null  int64         
 8   team_id        126314 non-null  object        
 9   fran_id        126314 non-null  object        
 10  pts            126314 non-null  int64         
 11  elo_i          126314 non-null  float64       
 12  elo_n          126314 non-null  float64       
 13  win_equiv      126314 non-null  float64       
 14  opp_id         126314 non-null  object        
 15  

In [ ]:
renamedNba = renamedNba.drop(columns=['notes'])
#or
#renamedNba.drop(['notes'],axis=1)

renamedNba.info()
print('-'*50)
renamedNba.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   gameorder      126314 non-null  int64         
 1   game_id        126314 non-null  object        
 2   lg_id          126314 non-null  object        
 3   _iscopy        126314 non-null  int64         
 4   year_id        126314 non-null  int64         
 5   date_game      126314 non-null  object        
 6   seasongame     126314 non-null  int64         
 7   is_playoffs    126314 non-null  int64         
 8   team_id        126314 non-null  object        
 9   fran_id        126314 non-null  object        
 10  pts            126314 non-null  int64         
 11  elo_i          126314 non-null  float64       
 12  elo_n          126314 non-null  float64       
 13  win_equiv      126314 non-null  float64       
 14  opp_id         126314 non-null  object        
 15  

### Changing the Data Type of Columns

In [ ]:
df = nba.copy()
df.info()
#df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126314 entries, 0 to 126313
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   gameorder      126314 non-null  int64         
 1   game_id        126314 non-null  object        
 2   lg_id          126314 non-null  object        
 3   _iscopy        126314 non-null  int64         
 4   year_id        126314 non-null  int64         
 5   date_game      126314 non-null  object        
 6   seasongame     126314 non-null  int64         
 7   is_playoffs    126314 non-null  int64         
 8   team_id        126314 non-null  object        
 9   fran_id        126314 non-null  object        
 10  pts            126314 non-null  int64         
 11  elo_i          126314 non-null  float64       
 12  elo_n          126314 non-null  float64       
 13  win_equiv      126314 non-null  float64       
 14  opp_id         126314 non-null  object        
 15  

*Convert column types*

In [ ]:
#object to datetime
df["date_played"] = pd.to_datetime(df["date_game"])

#pd.to_numeric
#astype(): str, int, float...

#dt.strftime()
#df["date_played"] = df["date_played"].dt.strftime('%d-%m-%Y')

*Identify unique values*

In [ ]:
df.head()

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes,date_played,DaysPassed,DaysCPassed
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,68,1300.0000,1306.7233,H,L,0.640065,NaN,01-11-1946,28555.0,28555.0
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,66,1300.0000,1293.2767,A,W,0.359935,NaN,01-11-1946,28555.0,28555.0
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,47,1306.7233,1297.0712,H,W,0.631101,NaN,02-11-1946,28554.0,28554.0
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,63,1300.0000,1309.6521,A,L,0.368899,NaN,02-11-1946,28554.0,28554.0
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,50,1300.0000,1320.3811,H,L,0.640065,NaN,02-11-1946,28554.0,28554.0


In [ ]:
a=df["game_location"].unique()
print(a)

['H' 'A' 'N']


*Counting distinct values*

In [ ]:
a=df["game_location"].nunique()
a

3

*Occurences*

In [ ]:
df['game_location'].value_counts()

,count
game_location,
H,63138
A,63138
N,38


In [ ]:
df['team_id'].value_counts()

,count
team_id,
BOS,5997
NYK,5769
LAL,5078
DET,4985
PHI,4533
...,...
INJ,60
PIT,60
DTF,60


Make colunms Category type

>Attention: check what are "categorial" values? Compare with strings or objects!

>Categorical data is often used for grouping and aggregating data

In [ ]:
nba.info()

In [ ]:
t= pd.Categorical(nba['team_id'] )

In [ ]:
t

['TRH', 'NYK', 'CHS', 'NYK', 'DTF', ..., 'CLE', 'GSW', 'CLE', 'CLE', 'GSW']
Length: 126314
Categories (104, object): ['ANA', 'AND', 'ATL', 'BAL', ..., 'WAT', 'WSA', 'WSB', 'WSC']

In [ ]:
df["game_location"] = pd.Categorical(df["game_location"])
df["game_location"].dtype

CategoricalDtype(categories=['A', 'H', 'N'], ordered=False, categories_dtype=object)

In [ ]:
df.info()

### Grouping features

Grouping allow to merge columns, applying aggregating functions: mean, average, sum, etc...

In [ ]:
nba.groupby("fran_id", sort=False)["pts"].sum()
# Expected:
# fran_id
# Huskies           3995
# Knicks          582497
# Stags            20398
# Falcons           3797
# Capitols         22387

,pts
fran_id,
Huskies,3995
Knicks,582497
Stags,20398
Falcons,3797
Capitols,22387


## 2 - Cleaning Data

Cleaning data means actions to overcome eventual existing problems with the data. It could be necessary to handle null values, duplicates, imbalanced datasets, outliers, etc.

In [ ]:
nba.info()

### Inputting Missing Values


Missing values can be handle by:
1. by removing (just that!)
2. by replacing (inputing):

* numeric values with "median" (if not normalized or have outliers)
* numeric values with "mean" (if normally distributed data)
* categorical values with "mode"


> **Note**: median is better than average because it is not susceptible to discrepancies in values. Otherwise, what would happen if a millionaire appeared in the average calculation?


Lets avoid *null-values*

The current nba dataset has null  values (*Null/None/ Nan Values*) (how to check that?).

The column "*notes*" has only 5424 *non-null* values. All remain columns have 126314 values..

(try to get this results)

Let analythe the following example:

In [ ]:
#import pandas as pd

data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 31, 25, None, 27],
        'Gender': ['F', 'M', None, 'M', 'F'],
        'Salary': [50000, None, 30000, 40000, 60000]}

df = pd.DataFrame(data)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    5 non-null      object 
 1   Age     4 non-null      float64
 2   Gender  4 non-null      object 
 3   Salary  4 non-null      float64
dtypes: float64(2), object(2)
memory usage: 288.0+ bytes


*It* is easy to realize that *Name* has 5  *non-null* values, but the other columns have only 4.

In [ ]:
#preserve original datatset
dfCopy = df.copy()
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


*Identify the Missing Values*

The missing values are converted by default. The functions to identify these missing values are:

*   **isnull()**
*   **notnull()**


The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.

"True" means the value is a missing value while "False" means the value is not a missing value.

In [ ]:
missing_data = dfCopy.isnull()
missing_data.head()

,Name,Age,Gender,Salary
0,False,False,False,False
1,False,False,False,True
2,False,False,True,False
3,False,True,False,False
4,False,False,False,False


### Replacing missing values

Replace null values of *Age* feature by *Unknown*

In [ ]:
dfCopy["Age"]= dfCopy["Age"].fillna(value="Unknown")

In [ ]:
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,Unknown,M,40000.0
4,Eva,27.0,F,60000.0


*Replace null values by a particular value*

In [ ]:
#dfCopy.fillna({'Age':'Unknown', 'Gender': 'Other'}, inplace=True)
#or
dfCopy=dfCopy.fillna({'Age':'Unknown', 'Gender': 'Other'})

In [ ]:
display(dfCopy)

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,Other,30000.0
3,David,Unknown,M,40000.0
4,Eva,27.0,F,60000.0


Sometimes NaN values are represented by "?". Replace the "?" symbol with *NaN* so the dropna() can remove the missing values:

In [ ]:
#set NaN values
df1=dfCopy.replace('?',np.NaN)

Fill **numerical features** with the *mean* value

In [ ]:
#reset dfCopy
dfCopy = df.copy()
dfCopy
#dfCopy.info()

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


In [ ]:
#Using mean() function to input the NaN values using fillna
dfCopy.fillna({'Salary':dfCopy['Salary'].mean()})

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,45000.0
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


In [ ]:
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,30000.0
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


Fill numerical features with the *mode* value

> mode() - most frequent value

In [ ]:
#checking...
dfCopy.mode()
#explain the result...

,Name,Age,Gender,Salary
0,Alice,25.0,F,30000.0
1,Bob,NaN,M,NaN
2,Charlie,NaN,NaN,NaN
3,David,NaN,NaN,NaN
4,Eva,NaN,NaN,NaN


In [ ]:
#Using mode() function to input the NaN values using fillna
#Why [0]?
dfCopy.fillna({'Salary':dfCopy['Salary'].mode()[0]}, inplace = True)
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,30000.0
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


Fill categorical features with the *mode* value

In [ ]:
mode=dfCopy['Age'].mode()[0] #mode()[0] gives the first mode if multiple exist
mode
#age_mode = dfCopy['Age'].mode()[0]
#age_mode

,Age
0,25.0


In [ ]:
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


In [ ]:
df3=dfCopy['Age'].fillna(mode)

In [ ]:
#Replace Age NaN values with the mode()
ag=dfCopy['Age'].mode()[0]
df2=dfCopy.fillna({'Age':ag, 'Gender':'Other'})
#or
df3=dfCopy.fillna({'Age':ag})
#dfCopy=dfCopy.fillna({'Age':dfCopy['Age'].mode()[0]})
#dfCopy.fillna({'Age':dfCopy['Age'].mode()[0]}, inplace = True)
#what does this?
df4=dfCopy['Age'].fillna(mode)
df4

In [ ]:
mode=dfCopy['Age'].mode()[0]
dfCopy=dfCopy.fillna({'Age':mode})
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,25.0,M,40000.0
4,Eva,27.0,F,60000.0


### See the *null* values

In [ ]:
#preserve original datatset
dfCopy = df.copy()
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


In [ ]:
n1 = dfCopy.isnull().any(axis=1)
n1

,0
0,False
1,True
2,True
3,True
4,False


### Get only the *null* values

In [ ]:
nullRows = dfCopy[n1]
nullRows

,Name,Age,Gender,Salary
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0


### Get only the *non-null* values

In [ ]:
n2 = dfCopy.notnull().all(axis=1)
n2

,0
0,True
1,False
2,False
3,True
4,True


In [ ]:
nonNullRows = dfCopy[n2]
nonNullRows

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
3,David,25.0,M,40000.0
4,Eva,27.0,F,60000.0


### Checking *Null Values* using Query Method

In this example, the != operator compare the column values with themselves, which returns *True* if the value is *null*.

In [ ]:
#preserve original datatset
dfCopy = df.copy()
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


In [ ]:
nullRows = dfCopy.query('Age != Age or Gender != Gender or Salary != Salary')

In [ ]:
nullRows

,Name,Age,Gender,Salary
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0


###  Remove rows with missing values

The easiest way to deal with records containing missing values (incomplete records) is to ignore them!


In [ ]:
#preserve original datatset
dfCopy = df.copy()
dfCopy

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
1,Bob,31.0,M,NaN
2,Charlie,25.0,None,30000.0
3,David,NaN,M,40000.0
4,Eva,27.0,F,60000.0


In [ ]:
dfCopy.shape
#dfCopy

(5, 4)

In [ ]:
#default axis=0 (index==rows)
rowsWithoutMissingData = dfCopy.dropna()

In [ ]:
rowsWithoutMissingData.shape

(2, 4)

In [ ]:
rowsWithoutMissingData

,Name,Age,Gender,Salary
0,Alice,25.0,F,50000.0
4,Eva,27.0,F,60000.0


### Remove *features* (columns) with null-values

Remove problematic columns if they’re not relevant for your analysis.

In [ ]:
#Features==Columns (axis 1)
dataWithoutMissingColumns = dfCopy.dropna(axis=1)

In [ ]:
dataWithoutMissingColumns

,Name
0,Alice
1,Bob
2,Charlie
3,David
4,Eva


### Change *Null Values*

In [ ]:
nba.info()

In [ ]:
#see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html
data_with_default_notes = nba.copy()
data_with_default_notes["notes"]=data_with_default_notes["notes"].fillna(value="no notes at all")
data_with_default_notes["notes"].describe()
# Expected:
# count              126314
# unique                232
# top       no notes at all
# freq               120890
# Name: notes, dtype: object

,notes
count,126314
unique,232
top,no notes at all
freq,120890


### Invalid Values

For instance, check when pts are zero (0)

In [ ]:
nba[nba["pts"] == 0]
#nba[nba["pts"] == 0]['pts']

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes,date_played,DaysPassed,DaysCPassed
26684,13343,197210260VIR,ABA,1,1973,10/26/1972,7,0,DNR,Nuggets,...,2,1484.1907,1487.083,A,L,0.328948,at Richmond VA; forfeit to VIR,26-10-1972,19064.0,19064.0


### Inconsistencies Between Values in Different Columns

In [ ]:
nba[(nba["pts"] > nba["opp_pts"]) & (nba["game_result"] != "W")].empty
# Expected:
# True

True

In [ ]:
nba[(nba["pts"] < nba["opp_pts"]) & (nba["game_result"] != "L")].empty
# Expected:
# True

True

## 3 - Outliers

### Understanding outliers

Analysing outliers, i.e., values very distant from the standard deviation.

See
* [How To Find Outliers in Data Using Python](https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/)
* [What Is an Outlier](https://careerfoundry.com/en/blog/data-analytics/what-is-an-outlier/#what-is-an-outlier)
* [How to Find Outliers in a Data Set](https://humansofdata.atlan.com/2017/10/how-to-find-outliers-data-set/)

Outliers are the extreme values within the dataset. They can be found in two ways:
* by using statistical calculs ( with standard deviation calculation).
* grapically (boxplot)

In [ ]:
#recover original df

data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 31, 25, None, 27],
        'Gender': ['F', 'M', None, 'M', 'F'],
        'Salary': [50000, None, 30000, 40000, 60000]}

df = pd.DataFrame(data)

In [ ]:
df



---



**A- Find outliers statisticaly - with *describe*()**

In [ ]:
df.describe()

In [ ]:
#all standard deviations
std_devs = df.std(numeric_only=True,ddof=1)
std_devs

In [ ]:
df.describe()['Salary']



---



**B - Find outliers statisticaly - with std()**

> To identify potential outliers using the standard deviation method, you can calculate the mean and standard deviation of a numeric column and check if any values lie outside the range defined by μ±kσ, where μ is the mean, σ is the standard deviation, and
k is a chosen threshold (commonly k=3).

Attention: To get better results, remember:

1. Remove NaN values
2. Sort the entire DataFrame by intended feature


Calculating the standard deviations with *std()* and *mean()*

In [ ]:
#Analyse only the Salary
desv = df['Salary'].std()
desv

In [ ]:
mean = df['Salary'].mean()
mean

In [ ]:
#find uncommon values, like greater 2 or 3 times (threshold) the std
df.loc[df['Salary']>= mean + 3 * desv, 'Salary'].count()
#no outliers

Another perspective:

In [ ]:
import numpy as np

# Calculate the mean and standard deviation for 'Salary' and 'Age'
for column in ['Salary', 'Age']:
    mean = df[column].mean()
    std_dev = df[column].std()
    threshold = 3  # Common threshold for outlier detection

    # Define the range for outliers
    lower_bound = mean - threshold * std_dev
    upper_bound = mean + threshold * std_dev

    # Identify outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    print(f"Analysis for {column}:")
    print(f"Mean: {mean}, Std Dev: {std_dev}")
    print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    print(f"Outliers:\n{outliers}\n")

Ok, according to std, we don't have outliers!

However, this approach works better for columns without missing values. For Salary and Age, missing values are ignored automatically by pandas.



---



**C - See it Graphically with BoxPLot and IRQ**


Steps:

1. Compute Quartiles:
* Q1: The 25th percentile (lower quartile).
* Q3: The 75th percentile (upper quartile).


2. Calculate IQR:
* IQR=Q3-Q1


3. Outlier Range:
* Lower Bound: Q1-1.5xIQR (very common to use 3)
* Upper Band: Q1+1.5*IQR. (very common to use 3)

4. Outliers:
* Any values below the lower bound or above the upper bound are considered outliers.

In [ ]:
df

In [ ]:
import plotly.express as px

In [ ]:
#create an horizontal box-plot
fig = px.box(df, x='Salary')
fig.update_xaxes(tickangle=-90)  # Rotate labels by -45 degrees
fig.update_layout(width=700, height=500)
fig.show()

In [ ]:
#create a box-plot
#fig = px.box(df, x='Salary')
fig = px.box(df, y='Salary', title="Vertical Box Plot of Salaries", labels={'Salary': 'Salary ($)'})
fig.update_layout(width=500, height=700)  # Specify width and height in pixels
fig.show()

#what this means
#fig = px.box(df, y='Salary', x='Age', title="Vertical Box Plot of Salaries", labels={'Salary': 'Salary ($)', 'Job': 'Job Role'})

In [ ]:
# or
# Create a boxplot for Salary
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 3))
plt.boxplot(df['Salary'].dropna(), vert=False, patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='blue'),
            whiskerprops=dict(color='blue'), capprops=dict(color='blue'),
            medianprops=dict(color='red'))

# Add labels and title
plt.title("Boxplot of Salary", fontsize=14)
plt.xlabel("Salary", fontsize=12)

# Display the plot
plt.show()

Boxes Analysis:

Boxplot Components:

* The box represents the interquartile range (IQR).
* The line inside the box represents the median.
* The whiskers extend to the smallest and largest data points within 1.5 x IQR from Q1 and Q3, respectively.
* Points outside the whiskers are plotted individually as outliers.

Expected Result:

* The salaries (30000, 40000, 50000, 60000) fall within the whiskers.
* No individual points outside the whiskers indicate no outliers in this dataset.

Final remarks:

* No points lie outside the whiskers, confirming there are no outliers in the Salary data

Another tools:

Using Scatter Plots:

In [ ]:

import matplotlib.pyplot as plt
# Scatter plot
plt.figure(figsize=(5, 6))
plt.scatter(df.index, df['Salary'], color='blue', label='Salary')
plt.axhline(y=df['Salary'].median(), color='red', linestyle='--', label='Median')
plt.title("Scatter Plot of Salary")
plt.xlabel("Index")
plt.ylabel("Salary")
plt.legend()
plt.grid(True)
plt.show()

Using a Histogram

In [ ]:
import matplotlib.pyplot as plt

# Calculate the median
median_value = df['Salary'].median()

# Histogram
plt.figure(figsize=(5, 6))
plt.hist(df['Salary'], bins=10, color='skyblue', edgecolor='black')
# Add a vertical line for the median
plt.axvline(median_value, color='red', linestyle='--', linewidth=2, label=f'Median: {median_value}')

# Labels, Title, and Legend
plt.title("Histogram of Salary with Median")
plt.xlabel("Salary")
plt.ylabel("Frequency")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()



---



*What happens if the the salary of Eva is 600000?*


In [ ]:
#recover original df

data2 = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 31, 25, None, 27],
        'Gender': ['F', 'M', None, 'M', 'F'],
        'Salary': [50000, None, 30000, 40000, 600000]}

df2 = pd.DataFrame(data2)
df2

Lets use IQR IQR=Q3-Q1)

In [ ]:
import pandas as pd

# Dataset
data2 = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 31, 25, None, 27],
    'Gender': ['F', 'M', None, 'M', 'F'],
    'Salary': [50000, None, 30000, 40000, 600000]
}

df2 = pd.DataFrame(data2)

# Drop NaN values and sort the entire DataFrame by 'Salary'
df_sorted = df2.dropna(subset=['Salary']).sort_values(by='Salary')

# Calculate Q1 (25th percentile) on the 'Salary' column
q1 = df_sorted['Salary'].quantile(0.25)
q3 = df2['Salary'].quantile(0.75)

# IQR=Q3−Q1
iqr=q3-q1

print("Q1=",q1)
print("Q3=",q3)
print("IRQ=",iqr)


In [ ]:
import plotly.express as px

# Box plot for Salary
fig = px.box(df2, y='Salary', title="Box Plot for Salary (Outlier Detection)", labels={'Salary': 'Salary ($)'})
fig.update_layout(width=500, height=700)  # Specify width and height in pixels
fig.show()

Remarks:
* no point after or before the wiskers.
* Thus, no outliers


**Final Explanation:**

* 600000 afects both, the std and the mean!

* Standard deviation works well if the data is normalized
* Standard deviation is sensitive to extreme values (outliers - tey distorce both μ (mean) and σ (standard deviation),
* Thus, std is less effective in datasets with large variability or skewed distributions.
* In these cases use more robust methods like ***Median Absolute Deviation (MAD)*** for outlier detection.



---



**Exercise**

What happens if the salaray values are:


'Salary': [50000, 40000, 30000, 70000, 1000000]





---



###Handling outliers

Analysing outliers, i.e., values very distant from the standard deviation.

See
* [How To Find Outliers in Data Using Python](https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/)
* [What Is an Outlier](https://careerfoundry.com/en/blog/data-analytics/what-is-an-outlier/#what-is-an-outlier)
* [How to Find Outliers in a Data Set](https://humansofdata.atlan.com/2017/10/how-to-find-outliers-data-set/)


Outliers are the extreme values within the dataset.

Let analyse a new dataset

**Get the dataset**



In [ ]:
filePath="/content/gDrive/MyDrive/Colab Notebooks/MIA - ML - 2024-2025/Datasets/"

In [ ]:
dataset = pd.read_csv(filePath+'credit_simple.csv', sep=';')
dataset.shape

In [ ]:
dataset.dtypes



---



**Preparing the dataset**

* Identify the dependent variable
* Isolate the feature "CLASSE"

In [ ]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1] #all raws, allcolumns from 0 to n-1

**Find outliers statisticaly - with *describe*()**

In [ ]:
X.describe()

In [ ]:

X.describe()['SALDO_ATUAL']
#Note:
# funtion describe() calculates the Sample standard deviation
# function std() calcules the Population standard deviation
# to make it equal, std(ddof=1)
# "ddof" stands for "Delta Degrees of Freedom"

Analysing it:

Statistic	| Value
----------|----------
count	| 993 (number of entries)
mean | 24,258,570 (average value)
std	| 688,349,600 (standard deviation)
min	| 250 (minimum value)
25%	| 1,371 (first quartile, lower 25%)
50%	| 2,323 (median, middle value)
75%	| 3,976 (third quartile, upper 25%)
max	| 21,544,410,000 (maximum value)

Notes:
1. Disparity Between std and Quartiles:

* The std (standard deviation) is 688,349,600, which is an extremely large value compared to the interquartile range (IQR), from 1,371 (25%) to 3,976 (75%).
* This suggests that the dataset has outliers or extreme values, especially near the maximum.

2. Skewness:

* The *mean* (24,258,570) is much larger than the *median* (2,323), indicating that the data is right-skewed, likely due to extreme high values (e.g., the maximum: 21,544,410,000).

Conclusion:

Considering SALDO_ATUAL, the minimum value (250.000) is very small compared with std (688,349,600). There are outliers, definitely!

In [ ]:
X['SALDO_ATUAL'].max()

In [ ]:
X.sort_values('SALDO_ATUAL',ascending=False)['SALDO_ATUAL']

In [ ]:
#import numpy as np

# Calculate the mean and standard deviation for 'SALDO_ATUAL'
column='SALDO_ATUAL'
mean = X[column].mean()
std_dev = X[column].std()
threshold = 3  # Common threshold for outlier detection

# Define the range for outliers
lower_bound = mean - threshold * std_dev
upper_bound = mean + threshold * std_dev

# Identify outliers
outliers = X[(X[column] < lower_bound) | (X[column] > upper_bound)]
#outliers = X[(X[column] > upper_bound)]

print(f"Analysis for {column}:")
print(f"Mean: {mean}, Std Dev: {std_dev}")
print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
print(f"Outliers:\n{outliers['SALDO_ATUAL']}\n")

There are two outliers!

**Find outliers statisticaly - with std()**

Calculating standard deviations with *std*()

In [ ]:
#all standard deviations
std_devs = X.std(numeric_only=True,ddof=1)
std_devs

In [ ]:
#Analyse only  SALDO_ATUAL
desv = X['SALDO_ATUAL'].std()
desv

In [ ]:
#find uncommon values, like greater 2 or 3 times the wiskers
X.loc[X['SALDO_ATUAL']> upper_bound, 'SALDO_ATUAL']
#there are two lines (127 and 160) that have such values

It is confirmed that SALDO_ATUAL has outliers (line 127 and 160)!

**Get outliers, graphicaly**

There are many methods for visualization and finding outliers in data:

* Histogram, Scatter Plot
* Box plot
* Scatter

See [Most Common Types of Data Visualization](https://careerfoundry.com/en/blog/data-analytics/data-visualization-types/)

In [ ]:
import plotly.express as px

In [ ]:
#create a box-plot
fig = px.box(X, x='SALDO_ATUAL')
fig.show()

Correcting Outliers

1. Removing affecting rows
2. Replacing values



Removing the affecting rows

In [ ]:
#X = X.drop([127,160])

In [ ]:
#X.info()

In [ ]:
#X.describe()

In [ ]:
print(filePath)

Let's handle outliers replacing by the median.

In [ ]:
#replace those values by tje median
mediana = X['SALDO_ATUAL'].median()
mediana

Replacing with *median*

In [ ]:
X.loc[X['SALDO_ATUAL']> upper_bound, 'SALDO_ATUAL']=mediana
#check again
X.loc[X['SALDO_ATUAL']> upper_bound,'SALDO_ATUAL'].count()


In [ ]:
#output the resultant DAtaframe
#X.to_csv(filePath+'newDataSet4.csv',columns=['SALDO_ATUAL'])

SALDO_ATUAL now, has no outliers!

In [ ]:
#create a box-plot
fig = px.box(X, x='SALDO_ATUAL')
fig.show()



---




Next Notebook will explore:

1. Data Bining
2. Categorical to Numeric
3. Datasets Manipulation

##4 - References

[Complete Guide to Feature Engineering: Zero to Hero](https://www.analyticsvidhya.com/blog/2021/09/complete-guide-to-feature-engineering-zero-to-hero/)

End!